In [ ]:
%load_ext abjad_notebook
import abjad

# Learning Abjad

## Beginner tutorial

In [ ]:
def slur_parts(voice, counts):
    notes = abjad.select.notes(voice)
    parts = abjad.select.partition_by_counts(notes, counts, cyclic=True)
    for part in parts:
        first_note, last_note = part[0], part[-1]
        accent = abjad.Articulation("accent")
        start_slur = abjad.StartSlur()
        abjad.attach(accent, first_note)
        abjad.attach(start_slur, first_note)
        staccato = abjad.Articulation("staccato")
        stop_slur = abjad.StopSlur()
        abjad.attach(staccato, last_note)
        abjad.attach(stop_slur, last_note)

In [ ]:
def tupletize_notes(voice, counts, modulus):
    notes = abjad.select.notes(voice)
    parts = abjad.select.partition_by_counts(notes, counts, cyclic=True)
    for i, part in enumerate(parts):
        if i % len(counts) == modulus:
            abjad.mutate.wrap(part, abjad.Tuplet("3:2"))

In [ ]:
tempo = 90

In [ ]:
string = r"c''16 f'' g'' a'' d'' g'' a'' b'' e'' a'' b'' c''' f'' b'' c''' d'''16"
voice_1 = abjad.Voice(string, name="Voice_1")
staff_1 = abjad.Staff([voice_1], name="Staff_1")

markup = abjad.Markup('\\markup "Cello I"')
instrument_name = abjad.InstrumentName(markup)
abjad.attach(instrument_name, voice_1[0])

mark = abjad.MetronomeMark((1, 4), tempo)
abjad.attach(mark, voice_1[0])

# abjad.show(staff_1)

In [ ]:
pitches = string.split()
pitches = reversed(pitches)
retrograde = " ".join(pitches)
voice_1.extend(retrograde)
# abjad.show(staff_1)

In [ ]:
string = string + " " + retrograde
voice_2 = abjad.Voice(string, name="Voice_2")
staff_2 = abjad.Staff([voice_2], name="Staff_2")

markup = abjad.Markup('\\markup "Cello II"')
instrument_name = abjad.InstrumentName(markup)
abjad.attach(instrument_name, voice_2[0])

mark = abjad.MetronomeMark((1, 4), tempo)
abjad.attach(mark, voice_2[0])

In [ ]:
score = abjad.Score([staff_1, staff_2], name="Score")
# abjad.show(score)

In [ ]:
for note in abjad.select.notes(voice_2):
    note.written_pitch = note.written_pitch.invert(axis="G4")

# abjad.show(score)

In [ ]:
slur_parts(voice_1, [2, 4, 4])
# abjad.show(score)

In [ ]:
slur_parts(voice_2, [4])
# abjad.show(score)

In [ ]:
tupletize_notes(voice_1, [6, 4], 0)
# abjad.show(score)

In [ ]:
tupletize_notes(voice_2, [4, 6], 1)
# abjad.show(score)

In [ ]:
del voice_1[-6:]
del voice_2[-3:]
# abjad.show(score)

In [ ]:
first_note = abjad.select.note(voice_1, 0)
time_signature = abjad.TimeSignature((2, 8))
abjad.attach(time_signature, first_note)
last_note = abjad.select.note(voice_2, -1)
bar_line = abjad.BarLine("|.")
abjad.attach(bar_line, last_note)
abjad.override(score).tuplet_bracket.staff_padding = 2
# abjad.show(score)

### Some additions to the tutorial

### TODO:

- Play inline midi
- Render notes event file

If I add midid block, note events are not generated.

In [ ]:
preamble = r"""#(ly:set-option 'relative-includes #t)
\include "./event-listener.ly"
"""

# score_block = abjad.Block(name="score")
# score_block.items.append(score)
# midi_block = abjad.Block(name="midi")
# score_block.items.append(midi_block)

lilypond_file = abjad.LilyPondFile([preamble, score])
!rm ./out*
abjad.persist.as_pdf(lilypond_file, "out.pdf")
abjad.show(score)

### Render with Zak

In [ ]:
from pathlib import Path

import numpy as np
import plotly.express as px
import soundfile as sf
import torch
from IPython.display import Audio

from performer.composition.score import parser
from performer.models.ddsp_module import DDSP

In [ ]:
flt_ckpt = "../checkpoints/flute_longrun.ckpt"
vln_ckpt = "../checkpoints/violin_longrun.ckpt"
vlc_ckpt = "../checkpoints/cello_longrun.ckpt"
drm_ckpt = "../checkpoints/drums_baseline.ckpt"
with torch.inference_mode():
    model = DDSP.load_from_checkpoint(flt_ckpt, map_location="cuda")
    model = model.to("cuda")
    model.eval()
    pass

In [ ]:
notes = parser("out-Cello I.notes")

start = 0
end = notes.notes[-1].t0 + notes.notes[-1].duration

t = np.linspace(start, end, int(250 * (end - start)))
env = notes.curve(t)
f0 = torch.from_numpy(notes.freq(t).astype("float32"))

env = torch.from_numpy(env.astype("float32"))
adsr = env * 90 - 100

with torch.inference_mode():
    p1 = f0[None, None, :].cuda()
    p2 = adsr[None, None, :].cuda()

    y = model(p1, p2)

y = y.cpu().squeeze().numpy()
sf.write("out.wav", y.T, 48000, "PCM_24")
Audio(data=y, rate=48000)

In [ ]:
px.line(env)

## Mono/Poly voices

In [ ]:
voice_1 = abjad.Voice(r"e''4 f''4 g''4 g''4 f''4 e''4 d''4 d''4 \fermata")
voice_2 = abjad.Voice(r"c''4 c''4 b'4 c''4 c''8 b'8 c''4 b'4 b'4 \fermata")
poly = abjad.Container([voice_1, voice_2], simultaneous=True)
literal = abjad.LilyPondLiteral(r"\voiceOne")
abjad.attach(literal, voice_1)
literal = abjad.LilyPondLiteral(r"\voiceTwo")
abjad.attach(literal, voice_2)

tail = abjad.Voice(r"c''4 c''4 b'4 c''4 c''8 b'8 c''4 b'4 b'4 \fermata")
staff = abjad.Staff([poly, tail], simultaneous=False)

abjad.show(staff)